In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import time

# plot
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
PATH = '/kaggle/input/hms-harmful-brain-activity-classification/'
df = pd.read_csv(PATH + 'train.csv')
TARGETS = df.columns[-6:]
print('Train shape:', df.shape )
print('Targets', list(TARGETS))
df.head()

In [ ]:
train = df.groupby('eeg_id')[['spectrogram_id','spectrogram_label_offset_seconds']].agg(
    {'spectrogram_id':'first','spectrogram_label_offset_seconds':'min'})
train.columns = ['spectrogram_id','min']

tmp = df.groupby('eeg_id')[['spectrogram_id','spectrogram_label_offset_seconds']].agg(
    {'spectrogram_label_offset_seconds':'max'})
train['max'] = tmp

tmp = df.groupby('eeg_id')[['patient_id']].agg('first')
train['patient_id'] = tmp

tmp = df.groupby('eeg_id')[TARGETS].agg('sum')
for t in TARGETS:
    train[t] = tmp[t].values
    
y_data = train[TARGETS].values
y_data = y_data / y_data.sum(axis=1,keepdims=True)
train[TARGETS] = y_data

tmp = df.groupby('eeg_id')[['expert_consensus']].agg('first')
train['target'] = tmp

train = train.reset_index()
print('Train non-overlapp eeg_id shape:', train.shape )
train.head(5)

In [ ]:
tmp

In [ ]:
train.iloc[2]

In [ ]:
import os
EEG_PATH = PATH + 'train_eegs/'

row = train.iloc[1]
eeg_id = row['eeg_id']
eeg_file_path = os.path.join(EEG_PATH, f'{eeg_id}.parquet')
sample_eeg_data = pd.read_parquet(eeg_file_path)
sample_eeg_data = sample_eeg_data.iloc[:, :-1]
sample_eeg_data = sample_eeg_data.iloc[:, [col for col in range(sample_eeg_data.shape[1]) if col not in [8, 9, 10]]]

# Calculate starting time point

start_time_point = int((sample_eeg_data.shape[0] - 10_000) // 2)

# Get the time point data of the middle 50 seconds

eeg_slice = sample_eeg_data.iloc[start_time_point : start_time_point + 10_000, :]

eeg_slice

In [ ]:
row = train.iloc[2]
eeg_id = row['eeg_id']
eeg_file_path = os.path.join(EEG_PATH, f'{eeg_id}.parquet')
sample_eeg_data = pd.read_parquet(eeg_file_path)
sample_eeg_data = sample_eeg_data.iloc[:, :-1]
sample_eeg_data = sample_eeg_data.iloc[:, [col for col in range(sample_eeg_data.shape[1]) if col not in [8, 9, 10]]]

# Calculate starting time point

start_time_point = int((sample_eeg_data.shape[0] - 10_000) // 2)

# Get the time point data of the middle 50 seconds

eeg_slice = sample_eeg_data.iloc[start_time_point : start_time_point + 10_000, :]

eeg_slice

In [ ]:
FLAG = True
if FLAG:
    fs = 200
    EEG_PATH = PATH + 'train_eegs/'

    # Initialize an array to save all processed EEG data
    all_eeg_data = np.zeros((len(train), 10000, 19))

    # Iterate over each eeg_id
    for row_idx in range(len(train)):
        row = train.iloc[row_idx]
        eeg_id = row['eeg_id']

        # Read EEG data
        eeg_file_path = os.path.join(EEG_PATH, f'{eeg_id}.parquet')
        eeg_data = pd.read_parquet(eeg_file_path)
        eeg_data = eeg_data.iloc[:, :-1]
#         eeg_data = eeg_data.iloc[:, [col for col in range(eeg_data.shape[1]) if col not in [8, 9, 10]]]
        # Calculate starting time point
        
        start_time_point = int((eeg_data.shape[0] - 10_000) // 2)

        # Get the time point data of the middle 50 seconds
        eeg_slice = eeg_data.iloc[start_time_point : start_time_point + 10_000, :]

        # Update all_eeg_data
        all_eeg_data[row_idx, :, :] = eeg_slice

    # The shape of the output array
    print("Shape of all_eeg_data:", all_eeg_data.shape)
else:
    print("FLAG is set to False. The code below is not executed.")

In [ ]:

#     Extract features from EEG data, including LL Spec, LP Spec, RP Spec, RL Spec
    
#     Parameters:
#     - eeg_data: EEG data with shape (number of samples, time points, number of electrodes)
    
#     Returns:
#     - Feature matrix with shape (number of samples, number of features)

import scipy.stats

def extract_eeg_features(eeg_data):
    # Initialize the feature matrix
    num_samples, num_time_points, num_electrodes = eeg_data.shape
    num_features = 8 * num_electrodes  # each electrode will have 8 quantity --> Mean, standard deviation, minimum, maximum, skewness, kurtosis, energy, entropy for each electrode + 4 global features
    features = np.zeros((num_samples, num_features))

    # Extract features
    for sample_idx in range(num_samples):
        for electrode_idx in range(num_electrodes):
            electrode_data = eeg_data[sample_idx, :, electrode_idx]
            feature_idx = electrode_idx * 8

            # Mean
            features[sample_idx, feature_idx] = np.mean(electrode_data)
            # Standard deviation
            features[sample_idx, feature_idx + 1] = np.std(electrode_data)
            # Minimum
            features[sample_idx, feature_idx + 2] = np.min(electrode_data)
            # Maximum
            features[sample_idx, feature_idx + 3] = np.max(electrode_data)
            # Skewness
            features[sample_idx, feature_idx + 4] = scipy.stats.skew(electrode_data)
            # Kurtosis
            features[sample_idx, feature_idx + 5] = scipy.stats.kurtosis(electrode_data)
            # Energy
            features[sample_idx, feature_idx + 6] = np.sum(electrode_data**2) / len(electrode_data)
            # Entropy
            features[sample_idx, feature_idx + 7] = scipy.stats.entropy(np.abs(electrode_data))
    return features

In [ ]:
import numpy as np
import scipy

from scipy.stats import skew, kurtosis
from scipy.stats.mstats import moment
from sklearn.preprocessing import StandardScaler

if FLAG:

    # Use the above functions to extract features
    extracted_features = extract_eeg_features(all_eeg_data)

    # The shape of the output feature matrix generated

    print("Shape of extracted features:", extracted_features.shape)
else:
    print("FLAG is set to False. The code below is not executed.")

In [ ]:
if FLAG:
    # Calculate the avg of each column
    mean_values = np.nanmean(extracted_features, axis=0)

    # Find the location of the NaN value

    nan_indices = np.isnan(extracted_features)

    # Replace NaN values with average

    extracted_features[nan_indices] = np.take(mean_values, nan_indices.nonzero()[1])

    # Output the feature shape after replacing NaN
    print("Shape of eeg_features after replacing NaN values:", extracted_features.shape)
    
    # Save extracted features in a file
    save_path = '/kaggle/working/extracted_eeg_features.npy'
    np.save(save_path, extracted_features)
    print(f"Extracted EEG features saved at: {save_path}")
else:
    # When FLAG is False, load the previously saved extracted EEG features.
    
    extracted_features = np.load('/kaggle/input/8-basic-feaatures-with-eeg/extracted_eeg_features (1).npy')

    # The shape of the output array

    print("Shape of extracted features:", extracted_features.shape)

In [ ]:
train_copy = train.copy()
ycol = [c for c in train_copy.columns if c in ['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']]
cd = {'Seizure':'seizure_vote', 'GPD':'gpd_vote', 'LRDA':'lrda_vote', 'Other':'other_vote', 'GRDA':'grda_vote', 'LPD':'lpd_vote'}
train_copy['target'] = train_copy['target'].map(cd)
for i in range(len(train_copy)):
    c = train_copy['target'][i]
    train_copy[c][i] = train_copy[c][i]+10 #adding weight to expert consensus

ysum = train_copy[ycol].sum(axis=1) 
for c in ycol:
    train_copy[c] = (train_copy[c] / ysum).astype(np.float64)

In [ ]:
label = train_copy[ycol]
label.shape

In [ ]:
train_copy.to_csv("final_train.csv")

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier

In [ ]:
X = extracted_features
y_labels = np.argmax(label.values, axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_labels, test_size=0.2, random_state=42)

In [ ]:
# Initialize the XGBoost classifier and specify tree_method as "gpu_hist"-->
xgb_clf = XGBClassifier(n_estimators=10000,
                        max_depth=6,
                        learning_rate=1e-1,
                        objective='multi:softmax',
                        num_class=len(np.unique(y_labels)),
                        early_stopping_rounds=100,
                        verbose=100,
                        tree_method='gpu_hist')

In [ ]:
# Training
xgb_clf.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=100)

In [ ]:
acc_xgb = accuracy_score(y_test, xgb_clf.predict(X_test))

In [ ]:
y_pred_xgb = xgb_clf.predict(X_test)

In [ ]:
# Output model accuracy
accuracy = accuracy_score(y_test, y_pred_xgb)
print(f"Accuracy: {accuracy}")

In [ ]:
from sklearn.metrics import mutual_info_score
kl = mutual_info_score(y_test,y_pred_xgb)

In [ ]:
kl

In [ ]:
test = pd.read_csv('/kaggle/input/hms-harmful-brain-activity-classification/test.csv')
print('Test shape',test.shape)
test.head()

In [ ]:
# FEATURE ENGINEER TEST
PATH2 = '/kaggle/input/hms-harmful-brain-activity-classification/test_eegs/'
data_test = np.zeros((len(test), extracted_features.shape[1]))
test_eeg = np.zeros((len(test), 10000, 19))

for k in range(len(test)):
    row = test.iloc[k]
    s = int(row.eeg_id)
    eeg_test = pd.read_parquet(f'{PATH2}{s}.parquet')
    eeg_test = eeg_test.iloc[:, :-1]
    #eeg_test = eeg_test.iloc[:, [col for col in range(eeg_test.shape[1]) if col not in [8, 9, 10]]]
    
    start_time_point = int((eeg_test.shape[0] - 10_000) // 2)

    eeg_test_slice = eeg_test.iloc[start_time_point : start_time_point + 10_000, :]
    
    test_eeg[k, :, :] = eeg_test_slice

    
    features_test = extract_eeg_features(test_eeg)
    print("Shape of features_test:", features_test.shape)

    data_test = features_test


print("Shape of data_test:", data_test.shape)

In [ ]:
# FEATURE ENGINEER TEST
PATH2 = '/kaggle/input/hms-harmful-brain-activity-classification/test_eegs/'
data_test = np.zeros((len(test), extracted_features.shape[1]))
test_eeg = np.zeros((len(test), 10000, 19))

for k in range(len(test)):
    row = test.iloc[k]
    s = int(row.eeg_id)
    eeg_test = pd.read_parquet(f'{PATH2}{s}.parquet')
    eeg_test = eeg_test.iloc[:, :-1]
    #eeg_test = eeg_test.iloc[:, [col for col in range(eeg_test.shape[1]) if col not in [8, 9, 10]]]
    # Calculate starting time point
    start_time_point = int((eeg_test.shape[0] - 10_000) // 2)
    
    # Get the time point data of the middle 50 seconds
    
    eeg_test_slice = eeg_test.iloc[start_time_point : start_time_point + 10_000, :]
    
    test_eeg[k, :, :] = eeg_test_slice

    # Use the previously defined extract_eeg_features function for feature extraction
    
    features_test = extract_eeg_features(test_eeg)
    print("Shape of features_test:", features_test.shape)
    # Put the extracted features into data_test
    data_test = features_test

# Output the shape of data_test

print("Shape of data_test:", data_test.shape)

In [ ]:
# Use the trained model to make predictions
predictions_proba = xgb_clf.predict_proba(data_test)

# Category name
class_names = ['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']

# Create a DataFrame to store probability values
probabilities_df = pd.DataFrame(predictions_proba, columns=class_names)

In [ ]:
probabilities_df.head()

In [ ]:
sub = pd.DataFrame({'eeg_id':test.eeg_id.values})
sub[TARGETS] = probabilities_df
sub.to_csv('submission.csv',index=False)
print('Submissionn shape',sub.shape)
sub.head()